In [ ]:
# Testen, warum der Check in meinem Test zur evaluation nicht funktioniert - Python rundet bescheiden
from pandas import DataFrame

def evaluation(df_measured, df_simulated_means):
    """Evaluates how much a simulations differs from measured data."""
    iterated_sum = 0
    absolutes_sum = 0
    for col in df_measured.columns:
        for ind in df_measured.index:
            print(col, ind)
            difference = df_measured[col][ind]-df_simulated_means[col][ind]
            print(difference)
            iterated_sum += difference
            absolutes_sum += abs(difference)
    
    return (iterated_sum, absolutes_sum)

df_measured = DataFrame({'PML I WT':[1.0,0.09,0.16,0.21,0.26,0.29,0.32]},index=[(-20,-10),(-10,10),(10,30),(30,50),(50,70),(70,90),(90,100)])
df_simulation = DataFrame({'PML I WT':[1.0,0.1,0.15,0.2,0.25,0.3,0.35]},index=[(-20,-10),(-10,10),(10,30),(30,50),(50,70),(70,90),(90,100)])
res = evaluation(df_measured, df_simulation)
print(res, res == (-0.02,0.08))

In [ ]:
import numpy as np
for i in range(5):
    np.random.seed(i)
    print(np.random.random())

In [ ]:
# einfach mal irgendeine Ratenkonstante berechnen
d1, d2 = 2, 3
m1, m2 = 0.5, 0.8
T = 298  # temperature [°K]
k = 1.380649*10**(-23)  # Boltzmann constant

d12 = (d1 + d2) / 2
m12 = (m1 * m2) / (m1 + m2)
v12 = np.sqrt( (8*k*T) / (np.pi*m12) ) # diffusion rate
c = np.pi*d12**2*v12

print(c)

In [ ]:
%matplotlib inline

from context import *
from pandas import DataFrame
import Gillespie as Glp
import matplotlib.pyplot as plt
import numpy as np

# initialise constants
species = ["PML_inner_fluorescent","PML_outer_fluorescent","PML_inner","PML_outer"]
# loss side
L = DataFrame({"reaction1":[1, 0, 0, 0],  # inner_f - outer_f
            "reaction2":[0, 1, 0, 0],  # outer_f - inner_f
            "reaction3":[0, 0, 1, 0],  # inner - outer
            "reaction4":[0, 0, 0, 1]},  # outer - inner
            index = species)
# gain side
R = DataFrame({"reaction1":[0, 1, 0, 0],
            "reaction2":[1, 0, 0, 0],
            "reaction3":[0, 0, 0, 1],
            "reaction4":[0, 0, 1, 0]}, index = species)
N = R - L

# Do reaction 1 and 2 have nearly the same rate constants?
rateConstants = {"reaction1": 0.06, "reaction2": 0.06, "reaction3": 0.04, "reaction4": 0.04}

startQuantities = {"PML_inner_fluorescent":[0],"PML_outer_fluorescent":[0],"PML_inner":[200],"PML_outer":[300]}

reaction_maximum = 5000
runs = 50

colours = {"PML_inner_fluorescent":"red", "PML_outer_fluorescent":"forestgreen", "PML_inner":"brown", "PML_outer":"darkgreen"}

In [ ]:
gillespies = Glp.monte_carlo_gillespie(rateConstants, L, N, startQuantities, runs=runs, reaction_limit=reaction_maximum)

In [ ]:
Glp.analyte_plot(gillespies, colours=colours)

In [ ]:
timesteps=[]
for x in range(1, len(gillespies[0].times)):
    timesteps.append(gillespies[0].times[x]-gillespies[0].times[x-1])
plt.plot(timesteps)

## Das Ganze ein wenig komplexer

Ich betrachte erstmal nur alle fluoreszierenden Moleküle und lasse diese in und aus den PMLs, sowie dann weiter auch in und aus der ROI wandern.

In [ ]:
# initialise constants
species2 = ["PML_inner_fluorescent","PML_outer_fluorescent","free_fluorescent"]
# loss side
L2 = DataFrame({"reaction1":[1, 0, 0],  # inner_f - outer_f
            "reaction2":[0, 1, 0],  # outer_f - inner_f
            "reaction3":[0, 1, 0],  # outer_f - free_f
            "reaction4":[0, 0, 1],  # free_f - outer_f
            "reaction5":[0, 0, 1],  # free_f - Nothing
            "reaction6":[0, 0, 0]},  # Nothing - free_f
            index = species2)
# gain side
R2 = DataFrame({"reaction1":[0, 1, 0],
            "reaction2":[1, 0, 0],
            "reaction3":[0, 0, 1],
            "reaction4":[0, 1, 0],
            "reaction5":[0, 0, 0],
            "reaction6":[0, 0, 1]}, index = species2)
N2 = R2 - L2

# Do reaction 1 and 2 have nearly the same rate constants?
rateConstants2 = {"reaction1": 0.04, "reaction2": 0.04, "reaction3": 0.06, "reaction4": 0.06, "reaction5": 0.8, "reaction6": 0.1}

startQuantities2 = {"PML_inner_fluorescent":[0],"PML_outer_fluorescent":[0],"free_fluorescent":[0]}

reaction_maximum = 5000
runs = 50

colours2 = {"PML_inner_fluorescent":"red", "PML_outer_fluorescent":"forestgreen", "free_fluorescent":"lightblue"}

In [ ]:
gillespies2 = Glp.monte_carlo_gillespie(rateConstants2, L2, N2, startQuantities2, runs=21, reaction_limit=reaction_maximum)

In [ ]:
plt.plot(gillespies2[0].times[:200], gillespies2[0].quantities["PML_inner_fluorescent"][:200])
plt.xlabel("time [rel]")
plt.ylabel("amount of molecules")

Das System, welches nur die fluoreszierenden Moleküle betrachtet scheint den FRAP-Vorgang nicht gut abzubilden. Ich nehme die gebleichten Moleküle mit rein.
Dabei werde ich erstmal ein möglichst einfaches System bauen (in ROI und außerhalb) und mich dann in der Komplexität steigern.

In [ ]:
species21 = ["ROI_fluorescent","outside_fluorescent","ROI","outside"]

L21 = DataFrame({"reaction1":[1, 0, 0, 0],
                 "reaction2":[0, 1, 0, 0],
                 "reaction3":[0, 0, 1, 0],
                 "reaction4":[0, 0, 0, 1]}, index = species21)
R21 = DataFrame({"reaction1":[0, 1, 0, 0],
                 "reaction2":[1, 0, 0, 0],
                 "reaction3":[0, 0, 0, 1],
                 "reaction4":[0, 0, 1, 0]}, index = species21)
N21 = R21 - L21

rateConstants21 = {"reaction1":2,
                   "reaction2":2,
                   "reaction3":2,
                   "reaction4":2}
startQuantities21 = {"ROI_fluorescent":[0], "outside_fluorescent":[100000], "ROI":[5000], "outside":[100000]}

reaction_maximum21 = 25000
runs21 = 6

colours21 = {"ROI_fluorescent":"red", "outside_fluorescent":"forestgreen", "ROI":"brown", "outside":"green"}

In [ ]:
gillespies21 = Glp.monte_carlo_gillespie(rateConstants21, L21, N21, startQuantities21, runs=runs21, reaction_limit=reaction_maximum21)

In [ ]:
gillespies21[0].plot(colours=colours21)
plt.plot(gillespies21[0].times, gillespies21[0].quantities["ROI"], "brown")
# plt.plot(gillespies21[0].times, gillespies21[0].quantities["outside"], "green")
plt.plot(gillespies21[0].times, gillespies21[0].quantities["ROI_fluorescent"], "red")

#print(gillespies21[0].quantities)

Wenn ich dieses einfache System simuliere, konvergiert das System nur, wenn die außerhalb der ROI liegenden Moleküle begrenzt sind. Wenn ich annehme, wie von Peter Dittrich  vorgeschlagen, dass außerhalb der ROI so viele Moleküle vorliegen, dass wir einen unerschöpflichen Vorrat annehmen können, dann wird dieses System so wie es dort steht nicht konvergieren.

### Wird das anders, wenn ich die einzelnen h-Werte endlich mal korrekt berechne?

In [ ]:
# initialise constants
species3 = ["PML_inner_fluorescent","PML_outer_fluorescent","free_fluorescent","PML_inner","PML_outer","free"]
# loss side
L3 = DataFrame({"reaction01":[1, 0, 0, 0, 0, 0],  # inner_f - outer_f
            "reaction02":[0, 1, 0, 0, 0, 0],  # outer_f - inner_f
            "reaction03":[0, 1, 0, 0, 0, 0],  # outer_f - free_f
            "reaction04":[0, 0, 1, 0, 0, 0],  # free_f - outer_f
            "reaction05":[0, 0, 1, 0, 0, 0],  # free_f - Nothing
            "reaction06":[0, 0, 0, 0, 0, 0],  # Nothing - free_f
            "reaction11":[0, 0, 0, 1, 0, 0],  # inner - outer
            "reaction12":[0, 0, 0, 0, 1, 0],  # outer - inner
            "reaction13":[0, 0, 0, 0, 1, 0],  # outer - free
            "reaction14":[0, 0, 0, 0, 0, 1],  # free - outer
            "reaction15":[0, 0, 0, 0, 0, 1],  # free - Nothing
            "reaction16":[0, 0, 0, 0, 0, 0]},  # Nothing - free
            index = species3)
# gain side
R3 = DataFrame({"reaction01":[0, 1, 0, 0, 0, 0],
            "reaction02":[1, 0, 0, 0, 0, 0],
            "reaction03":[0, 0, 1, 0, 0, 0],
            "reaction04":[0, 1, 0, 0, 0, 0],
            "reaction05":[0, 0, 0, 0, 0, 0],
            "reaction06":[0, 0, 1, 0, 0, 0],
            "reaction11":[0, 0, 0, 0, 1, 0],
            "reaction12":[0, 0, 0, 1, 0, 0],
            "reaction13":[0, 0, 0, 0, 0, 1],
            "reaction14":[0, 0, 0, 0, 1, 0],
            "reaction15":[0, 0, 0, 0, 0, 0],
            "reaction16":[0, 0, 0, 0, 0, 1]}, index = species3)
N3 = R3 - L3

# Do reaction 1 and 2 have nearly the same rate constants?
rateConstants3 = {"reaction01": 0.04,
                  "reaction02": 0.04,
                  "reaction03": 0.06,
                  "reaction04": 0.06,
                  "reaction05": 0.8,
                  "reaction06": 0.1,
                  "reaction11": 0.04,
                  "reaction12": 0.04,
                  "reaction13": 0.06,
                  "reaction14": 0.06,
                  "reaction15": 0.8,
                  "reaction16": 0.1}

startQuantities3 = {"PML_inner_fluorescent":[0],"PML_outer_fluorescent":[0],"free_fluorescent":[0],"PML_inner":[100],"PML_outer":[200],"free":[5000]}

reaction_maximum = 7000
runs3 = 21

colours3 = {"PML_inner_fluorescent":"red", "PML_outer_fluorescent":"forestgreen", "free_fluorescent":"lightblue", "PML_inner":"brown", "PML_outer":"green", "free":"blue"}

In [ ]:
gillespies3 = Glp.monte_carlo_gillespie(rateConstants3, L3, N3, startQuantities3, runs=runs3, reaction_limit=reaction_maximum)

In [ ]:
Glp.multiplot(gillespies3[:12], colours=colours3)

In [ ]:
#plt.plot(gillespies3[0].times,gillespies3[0].quantities["PML_inner_fluorescent"],"red")
#plt.plot(gillespies3[0].times,gillespies3[0].quantities["PML_outer_fluorescent"],"forestgreen")
#plt.plot(gillespies3[0].times,gillespies3[0].quantities["PML_inner"],"brown")
#plt.plot(gillespies3[0].times,gillespies3[0].quantities["PML_outer"],"green")
plt.plot(gillespies3[0].times,gillespies3[0].quantities["free_fluorescent"],"lightblue")
plt.plot(gillespies3[0].times,gillespies3[0].quantities["free"],"blue")

In [ ]:
# plt.plot(gillespies3[0].times,sum(x for x in gillespies3[0].quantities))

In [ ]:
gill = gillespies3[0]
Summe = []
for i in range(len(gill.times)):
    asum = 0
    for key in ["PML_inner_fluorescent","PML_outer_fluorescent","free_fluorescent"]:
            asum += gill.quantities[key][i]
    Summe.append(asum)
plt.plot(gill.times, Summe)

Die Reaktion mit dem Aus-dem-Nichts-erschaffen funtkioniert im Gillespie nicht, weil sich die Wahrscheinlichkeit des Stattfindens einer Reaktion aus der Anzahl der teilnehmenden Molküle berechnet, da aber keine Moleküle teilnehmen, sondern nur welche entstehen, wird die Reaktion nie stattfinden.

Ich nehme daher an, dass das System randvoll mit Molekülen ist und dass in dem Fall, dass ein "free" nach außen getragen wird gleichzeitg ein anderes "free" oder "free_fluorescent" in die ROI eintritt, diese Spezies sich also ineinander umwandeln.

In [ ]:
# initialise constants
species3 = ["PML_inner_fluorescent","PML_outer_fluorescent","free_fluorescent","PML_inner","PML_outer","free"]
# loss side
L4 = DataFrame({"reaction01":[1, 0, 0, 0, 0, 0],  # inner_f - outer_f
            "reaction02":[0, 1, 0, 0, 0, 0],  # outer_f - inner_f
            "reaction03":[0, 1, 0, 0, 0, 0],  # outer_f - free_f
            "reaction04":[0, 0, 1, 0, 0, 0],  # free_f - outer_f
            "reaction11":[0, 0, 0, 1, 0, 0],  # inner - outer
            "reaction12":[0, 0, 0, 0, 1, 0],  # outer - inner
            "reaction13":[0, 0, 0, 0, 1, 0],  # outer - free
            "reaction14":[0, 0, 0, 0, 0, 1],  # free - outer
            "reaction15":[0, 0, 0, 0, 0, 1],  # free - free_f
            "reaction16":[0, 0, 1, 0, 0, 0]},  # free_f - free
            index = species3)
# gain side
R4 = DataFrame({"reaction01":[0, 1, 0, 0, 0, 0],
            "reaction02":[1, 0, 0, 0, 0, 0],
            "reaction03":[0, 0, 1, 0, 0, 0],
            "reaction04":[0, 1, 0, 0, 0, 0],
            "reaction11":[0, 0, 0, 0, 1, 0],
            "reaction12":[0, 0, 0, 1, 0, 0],
            "reaction13":[0, 0, 0, 0, 0, 1],
            "reaction14":[0, 0, 0, 0, 1, 0],
            "reaction15":[0, 0, 1, 0, 0, 0],
            "reaction16":[0, 0, 0, 0, 0, 1]}, index = species3)
N4 = R4 - L4

# Do reaction 1 and 2 have nearly the same rate constants?
rateConstants4 = {"reaction01": 0.04,
                  "reaction02": 0.04,
                  "reaction03": 0.06,
                  "reaction04": 0.06,
                  "reaction11": 0.04,
                  "reaction12": 0.04,
                  "reaction13": 0.06,
                  "reaction14": 0.06,
                  "reaction15": 0.1,
                  "reaction16": 0.1}

startQuantities3 = {"PML_inner_fluorescent":[0],"PML_outer_fluorescent":[0],"free_fluorescent":[0],"PML_inner":[100],"PML_outer":[200],"free":[5000]}

reaction_maximum = 10000
runs3 = 21

colours3 = {"PML_inner_fluorescent":"red", "PML_outer_fluorescent":"forestgreen", "free_fluorescent":"lightblue", "PML_inner":"brown", "PML_outer":"green", "free":"blue"}

In [ ]:
gillespies4 = Glp.monte_carlo_gillespie(rateConstants4, L4, N4, startQuantities3, runs=runs3, reaction_limit=reaction_maximum)

In [ ]:
Glp.multiplot(gillespies4[:12], colours=colours3)

Die Kurven zeigen sehr gut, wie in den einzelnen Bereichen die Molekülpopulationen wachsen oder schwinden. Nur das Gesamtleuchten geht aus diesen Abbildungen noch nicht hervor.

In [ ]:
plt.plot(gillespies4[0].times, gillespies4[0].quantities["PML_inner"],"brown")
plt.plot(gillespies4[0].times, gillespies4[0].quantities["PML_inner_fluorescent"], "red")

Beide PML_inner-Populationen steigen, was heißt, dass im Inneren zu Beginn zu wenig Moleküle für die angegebenen Ratenkonstanten vorhanden waren. Oder die Ratenkonstanten müssen angepasst werden.

In [ ]:
Summe2 = []
for i in range(len(gillespies4[0].times)):
    bsum = 0
    for key in ["PML_inner_fluorescent","PML_outer_fluorescent","free_fluorescent"]:
        bsum += gillespies4[0].quantities[key][i]
    Summe2.append(bsum)
plt.plot(gillespies4[0].times, Summe2)

Das sieht doch schon besser aus.

In [ ]:
# initialise constants
species_lenser = ["x","y","z"]  # z inner und x free, y loosely bound
# loss side
L_lenser = DataFrame({"reaction01":[0, 0, 0],  # außen - x
            "reaction02":[1, 0, 0],  # x - y
            "reaction03":[0, 1, 0],  # y - z
            "reaction11":[1, 0, 0],  # x - außen
            "reaction12":[0, 1, 0],  # y - x
            "reaction13":[0, 0, 1]},  # z - y
            index = species_lenser)
# gain side
R_lenser = DataFrame({"reaction01":[1, 0, 0],  # außen - x
            "reaction02":[0, 1, 0],  # x - y
            "reaction03":[0, 0, 1],  # y - z
            "reaction11":[0, 0, 0],  # x - außen
            "reaction12":[1, 0, 0],  # y - x
            "reaction13":[0, 1, 0]},  # z - y
            index = species_lenser)
N_lenser = R_lenser - L_lenser

# Do reaction 1 and 2 have nearly the same rate constants?
rateConstants_lenser = {"reaction01": 1,
                  "reaction02": 1,
                  "reaction03": 1,
                  "reaction11": 0.1,
                  "reaction12": 1,
                  "reaction13": 1}

startQuantities_lenser = {"x":[0],"y":[0],"z":[0]}

reaction_max_lenser = 20000
runs_lenser = 2

colours_lenser = {"x":"red", "y":"forestgreen", "z":"lightblue"}

In [ ]:
gillespies_lenser = Glp.monte_carlo_gillespie(rateConstants_lenser, L_lenser, N_lenser, startQuantities_lenser, runs=runs_lenser, reaction_limit=reaction_max_lenser)

In [ ]:
Glp.multiplot(gillespies_lenser, colours=colours_lenser)
Glp.analyte_plot(gillespies_lenser, colours=colours_lenser)